## Inactivity_time UNIT화 하기! 함수 만들기

### input으로 받아야 할 것 : file이름, inactivity time을 직접 입력할 것인지 여부, 직접 한다면 어떤 값으로 할것인지? 비율로 입력한다면 몇 퍼센트?

### output은 csv file이겠지유

In [27]:
def pathmaker_inactivity(input_filename,output_filename, inactivity_setting, inactivity_time):
    import pandas as pd
    import numpy as np
    import datetime
    from tqdm import tqdm
    
    df = pd.read_csv(input_filename) ## filename은 'union_1908.csv' 이런 식이어야 한다!
    
    df['partner_key'].fillna('null', inplace = True)
    df['amount'].fillna(0, inplace = True)
    
    ##이 partner dic을 함수의 input으로 받을 것인지는 생각!!
    partner_dic = {'T1m2A6iwPUemMh5gQZ3FRA':'a', 'NGHO984OM0SciKycDGPmYA':'b', 'RK10BMLBB02TTT0G0GWMrg':'c', '0BChBVxZSUOhg3yJgn7o0A':'d', '41loF2NK40GBUzUBDC1Zrw':'e', 'o72f0c4oF0WMsPGvuIt5vQ':'f', 'null':'direct'}
    
    for key, value in partner_dic.items():
        df.loc[df['partner_key'] == key, 'index'] = value
    
    
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['timediff'] = df.datetime.diff().dt.days*24 + (df.datetime.diff().dt.seconds / 3600)
    
    id_dic = {}
    for key, value in df.adid.items():
        id_dic[value] = key+1
    change = list(id_dic.values())[:-1]
    
    df.loc[change, 'timediff'] = np.nan
    
    ##inactivity_setting = 0 이면 비율로!! 1이면 사용자 직접 입력!
    if inactivity_setting == 0:
        inactivity = df.timediff.quantile(inactivity_time)
        
    elif inactivity_setting == 1:
        inactivity = inactivity_time
    else :
        print("Your 'inactivity_setting' input is wrong. Please input 0 for ratio, 1 for direct input.")
    
    
    df['micro_session'] = 0
    df.loc[0, 'micro_session'] = 1
    df.loc[change, 'micro_session'] = 1
    
    timediff_change = list(map(lambda x: int(1) if x >= inactivity else np.nan, list(df['timediff'])))
    timediff_change = [i for i in range(len(timediff_change)) if timediff_change[i] == 1]
    df.loc[timediff_change, 'micro_session'] = 1
    
    purchase_change = list(map(lambda x: int(1) if x == "abx:purchase" else np.nan, list(df['event_name'])))
    purchase_change = [i+1 for i in range(len(purchase_change)) if purchase_change[i] == 1]
    if purchase_change[-1] == len(df):
        purchase_change = purchase_change[:-1]
    df.loc[purchase_change, 'micro_session'] = 1
    
    ms = list(df['micro_session'])
    ix = list(df['index'])
    mt = list(df['amount'])

    path = []
    amount = []
    for p, i, m in zip(ms, ix, mt):
        if p == 1:
            path.append([i])
            amount.append([m])
        elif p == 0:
            path[-1].append(i)
            amount[-1].append(m)

    path = ['>'.join(p) for p in path]
    amount = [sum(m) for m in amount]
    conversion = [1 if p.endswith('direct') else 0 for p in path]
    null = [1 if c == 0 else 0 for c in conversion]

    fin_df = pd.DataFrame({'path': path, 'total_conversions': conversion, 'total_conversion_value': amount, 'total_null': null})

    pppp = [path[:-7] if '>direct' in path else path for path in list(fin_df['path'])]
    
    fin_df['path'] = pppp 
    fin_df2 = fin_df[fin_df.path !='direct']

    fin_df2.to_csv(output_filename, index = False)

In [28]:
pathmaker_inactivity('union_1908.csv','REAL FINAL.csv', 0, 0.75)